# Food Lookup Tool & Token Analysis

This notebook evaluates the search volume and token costs for various food keywords.
It uses a local `search_food_unlimited` function to understand the full scale of results in the database.

In [7]:
import sys
import os
from sqlalchemy import select
import pandas as pd

# Add src to path to import database config and models
sys.path.append(os.path.abspath('src'))

from src.database import get_db_session
from src.models import FoodItem

In [8]:
def search_food_unlimited(query: str):
    """
    Search function for analysis. Returns ALL results matching the query.
    """
    session = get_db_session()
    try:
        stmt = select(FoodItem.name).where(FoodItem.name.ilike(f"%{query}%"))
        results = session.execute(stmt).scalars().all()
        return list(results)
    finally:
        session.close()

def estimate_tokens(text_list):
    """
    Simple token estimation: sums the word count of all strings in the list.
    """
    total_words = 0
    for text in text_list:
        total_words += len(text.split())
    return total_words

In [9]:
test_keywords = [
    "Rice", 
    "White rice",
    "White bread", 
    "Tuna", 
    "Tuna in water",
    "Chicken thigh", 
    "Pasta"
]

analysis_data = []

for kw in test_keywords:
    all_results = search_food_unlimited(kw)
    count = len(all_results)
    tokens = estimate_tokens(all_results)
    
    analysis_data.append({
        "Keyword": kw, 
        "Total Results": count, 
        "Estimated Tokens": tokens
    })

analysis_df = pd.DataFrame(analysis_data)
analysis_df

,Keyword,Total Results,Estimated Tokens
0,Rice,5,29
1,White rice,0,0
2,White bread,0,0
3,Tuna,1,1
4,Tuna in water,0,0
5,Chicken thigh,0,0
6,Pasta,0,0


### Conclusion
Review the table above to understand the scale of passing all search results to an LLM. 
If 'Estimated Tokens' is high (e.g., > 1000), a more aggressive filtering or ranking strategy is needed before context insertion.


Because of the DB structure, I need to remember to add to the agent instructions to look up with generic names and check its results.